In [1]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [2]:
file_name =  "clean_data_2022-07-28.csv"
folder = "../clean_equity_data/"
save = "YES"

data_tmp = pd.read_csv(folder + file_name)

In [3]:
data_tmp.columns

Index(['Unnamed: 0', 'Börsdata ID', 'Company', 'Industry', 'Volume', 'P/FCF',
       'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Market Cap',
       'Volatility', 'ROE', 'OCF', 'Total Equity', 'EV', 'Return 6m',
       'Return 1y', 'Country', 'List', 'Tick', 'EV/EBIT', 'Last Report',
       'Yahoo', 'Yield', 'P/E', 'EV/S', 'P/B', 'Cash.f.Fin.',
       'Info - Stock price', 'Info - Report', 'Sector', 'CF_prof', 'GPA',
       'Financing_Yield', 'EA ret', 'EA ret std', '1 Year Volatility',
       '1 Year MAD', 'EA Volatility', 'Beta', 'Res_Mom_1Y', 'Res_Mom_6m',
       'Total_Mom_6m'],
      dtype='object')

In [4]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [5]:
### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Gestalt Imputation and Rank

In [6]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'EV/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'CF_prof', 'Financing_Yield']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'EV/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [7]:
RANK_WEIGHTS = {"Momentum": 1, "Value": 1, "Quality": 1}



factor=1.0/sum(RANK_WEIGHTS.values())
for k in RANK_WEIGHTS:
    RANK_WEIGHTS[k] = RANK_WEIGHTS[k]*factor
    
    
print(sum(RANK_WEIGHTS.values()))

1.0


In [8]:
####### CALCULATE RANKS ###########
for i in ['Total_Mom_6m', 'Return 6m', 'Return 1y', 'EA ret std', 'Res_Mom_1Y', 'Res_Mom_6m']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank(ascending = False)

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'EV/S', 'P/FCF', 'EV/EBIT', 'Financing_Yield']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'CF_prof', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (data_gestalt[i]).rank(ascending = False)
    
##################### Composite ranks #######################
data_gestalt['PMOM Rank'] = (data_gestalt['Total_Mom_6m Rank'] + data_gestalt['Return 1y Rank']).rank(ascending = True)
data_gestalt['IMOM Rank'] = (data_gestalt['Res_Mom_1Y Rank'] + data_gestalt['Res_Mom_6m Rank']).rank(ascending = True)
data_gestalt['Momentum Rank'] = (data_gestalt['EA ret std Rank'] + 
                                 data_gestalt['IMOM Rank'] + data_gestalt['PMOM Rank']).rank(ascending = True)


data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['CF_prof Rank'] + 
                        data_gestalt['GPA Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['EV/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Financing_Yield Rank']).rank()
 


    
data_gestalt['Gestalt Rank'] = ((RANK_WEIGHTS['Value']*data_gestalt['Value Rank'] 
                                 + RANK_WEIGHTS['Momentum']*data_gestalt['Momentum Rank'] +
                                RANK_WEIGHTS['Quality']* data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [9]:
#data_mom.isna().sum()
data_gestalt.isna().sum().sum()

51

### Sorting

In [10]:
CURRENT_BUYOUTS = ['Aspire Global', 'LeoVegas', 'Swedish Match', 'Atvexa']

#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])
gestalt_tmp=gestalt_tmp[ ~gestalt_tmp['Company'].isin(CURRENT_BUYOUTS)]


In [11]:

if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "GESTALT_{}".format(data_date.group(0))+ ".csv"
    gestalt_tmp.to_csv(os.path.join(folder, exp_file_name))

In [12]:
compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

stocks = 30


## TRIPLE SORT ##

gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

### Print Stock Selection

In [13]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 15+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'Gestalt Rank': "{:.0f}",'Quality Rank': "{:.0f}", 'Value Rank': "{:.0f}",
                          'Momentum Rank': "{:.0f}",
                          'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,Rottneros,Mid Cap,RROS,2,40,3,3,1.6,43.4%
2,Dedicare,Small Cap,DEDI,2,8,30,8,2.9,43.1%
3,Björn Borg,Small Cap,BORG,3,12,26,38,0.6,44.7%
4,SSAB B,Large Cap,SSAB B,4,41,2,37,-0.1,41.2%
5,Arctic Paper,Small Cap,ARP,5,74,4,6,1.3,53.0%
6,B3 Consulting,Small Cap,B3,6,10,47,35,-0.6,52.1%
7,Prevas,Small Cap,PREV B,8,16,32,48,0.0,50.7%
8,EnQuest,Mid Cap,ENQ,8,80,12,5,1.9,61.6%
9,Betsson,Mid Cap,BETS B,9,60,37,4,3.5,41.2%
10,BE Group,Small Cap,BEGR,10,66,8,31,0.7,66.2%


## Reversed Ranks

In [14]:
stocks = 20

## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

neg_gestalt = gestalt_tmp.tail(stocks)[::-1][compact]

In [17]:
np.sqrt(252)

15.874507866387544

In [21]:
gestalt_tmp.sort_values(by = 'EA ret std').tail(30)[['Company','Res_Mom_1Y', 'EA ret std', 'Financing_Yield','Momentum Rank']]


,Company,Res_Mom_1Y,EA ret std,Financing_Yield,Momentum Rank
297,Embracer,-14.513908,2.009495,0.267778,144.0
239,Cint Group,-2.757459,2.062487,0.345487,134.0
309,Stillfront,-17.568495,2.139603,0.117129,213.5
350,MedCap,4.306441,2.149546,-0.051291,18.0
128,Alfa Laval,-0.472682,2.154281,-0.035277,53.0
178,Hanza Holding,18.054975,2.168849,0.031061,9.0
265,Infrea,-9.014722,2.170219,0.072382,157.0
225,Truecaller,-2.757459,2.349742,0.366652,146.0
43,TietoEVRY,7.865624,2.420283,-0.073430,20.0
269,Lyko,-25.530859,2.429908,-0.011575,189.5


In [20]:
neg_gestalt

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
328,Senzime,Small Cap,SEZI,308.0,301.5,297.0,290.0,-1.603635,0.398853
299,ExpreS2ion Biotech,First North,EXPRS2,307.0,292.5,303.0,276.0,-0.216381,0.848453
248,Nanoform,First North,NANOFS,306.0,289.0,274.0,284.0,-0.749432,0.413868
331,OptiCept Technologies,First North,OPTI,305.0,276.0,268.0,293.5,-1.428968,0.583721
340,Cell Impact,First North,CI,304.0,285.0,288.0,257.0,-0.197534,0.818007
356,Episurf Medical,Small Cap,EPIS B,303.0,297.0,287.0,240.0,0.688558,0.594159
270,Acconeer,First North,ACCON,302.0,283.5,305.0,234.0,-0.668314,0.680575
257,Azelio,First North,AZELIO,301.0,291.0,229.0,298.0,-0.929730,1.067486
293,Smart Eye,First North,SEYE,300.0,269.0,237.0,308.0,-3.256216,0.616463
330,Enzymatica,First North,ENZY,299.0,272.0,271.0,266.0,-0.554100,0.928298


In [17]:

if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "rank_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))

In [62]:
company('EnQuest', gestalt_tmp)

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,ROC Rank,CF_prof Rank,GPA Rank,Assets Turn Rank,PMOM Rank,IMOM Rank,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
385,385,75,EnQuest,Oil & Gas -Exploration,23.673,1.813,0.29,3463.601,42215.672,0.18789,...,90.0,42.0,267.0,252.0,26.0,26.0,7.0,82.0,13.0,8.5


In [21]:
gestalt_tmp[['Momentum Rank', 'Quality Rank', 'Value Rank', 'Gestalt Rank']].corr(method = 'spearman')

,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
Momentum Rank,1.000000,0.223704,0.115454,0.602509
Quality Rank,0.223704,1.000000,0.570150,0.814957
Value Rank,0.115454,0.570150,1.000000,0.761501
Gestalt Rank,0.602509,0.814957,0.761501,1.000000


In [24]:


gestalt_tmp.loc[gestalt_tmp['Company'] == 'EnQuest',['Company','Financing_Yield Rank']]

,Company,Financing_Yield Rank
385,EnQuest,57.0


In [22]:
gestalt_tmp.sort_values(by = 'Financing_Yield Rank')

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,ROC Rank,CF_prof Rank,GPA Rank,Assets Turn Rank,PMOM Rank,IMOM Rank,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
212,212,2383,Momentum Group,Industrial Components,3.007,36.566,1.563,761.000,1017.000,0.26282,...,50.0,60.0,15.0,20.0,306.0,172.0,239.0,18.0,100.0,96.5
32,32,222,Tele2 B,Telecommunications,314.090,5.163,0.411,11047.000,66305.000,0.12344,...,134.0,44.0,225.0,221.0,78.0,56.0,114.0,96.0,76.0,63.0
88,88,383,G5 Entertainment,Gaming,13.193,15.845,1.952,840.692,677.301,0.58348,...,16.0,1.0,3.0,9.5,258.5,288.5,195.0,3.0,31.0,37.0
37,37,210,Softronic,IT Consulting,0.762,13.323,1.991,528.200,412.300,0.56498,...,19.0,14.0,2.0,7.0,57.5,68.0,102.0,5.0,57.0,19.5
280,280,1474,Bonesupport,Biotechnology,20.118,2635.405,0.811,230.121,317.063,-0.79409,...,290.0,283.0,17.0,124.0,57.5,98.0,66.0,230.0,247.0,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,1383,Gapwaves,Communications,0.708,2635.405,0.160,27.968,239.330,-1.24820,...,298.0,274.0,250.0,270.0,204.0,202.0,221.5,281.0,299.0,296.0
298,298,1361,Filo Mining,Mining,6.850,2635.405,0.718,-310.110,880.985,0.10460,...,154.5,277.0,306.0,154.5,8.0,60.5,62.5,266.0,291.0,230.0
289,289,1434,Isofol Medical,Pharmaceuticals,1.792,2635.405,0.060,NaN,350.566,0.10460,...,154.5,299.0,154.5,285.0,186.0,105.0,181.0,239.5,297.0,270.0
283,283,1456,ISR Holding,Biotechnology,2.271,2635.405,0.718,-56.291,56.968,0.10460,...,154.5,305.0,308.0,154.5,118.0,130.0,137.0,279.0,289.0,268.0
